In [35]:
import urllib.request, json
import pandas as pd
import numpy as np
import os

In [33]:
os.getcwd()

'C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\판매량'

In [26]:
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\판매량')
data =open('final_all_sell(year).json').read()
data = pd.DataFrame(json.loads(data))

In [27]:
PUP_C  = data[data['GLOBAL_SEGMENT'] =='PUP-C']
PUP_C = PUP_C[PUP_C['Y2016'] != 0]
PUP_C

,Brand,GLOBAL_MODEL,GLOBAL_SEGMENT,Strategic Group,Y2012,Y2013,Y2014,Y2015,Y2016
237,Toyota,Tacoma,PUP-C,Toyota,141354,159485,155042,179564,191632
33,Chevrolet,Colorado,PUP-C,GM,36840,3412,8003,84430,108725
357,Honda,Ridgeline,PUP-C,Honda,14068,17723,13389,520,23668
399,Nissan,Frontier,PUP-C,Renault-Nissan,55435,62837,74323,62817,86926
54,GMC,Canyon,PUP-C,GM,8735,929,3070,30077,37449


# 수집할 feature 정하기

# SPEC 수집하기

In [36]:
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\spec')

In [37]:
pre_dummy = pd.DataFrame(np.arange(1).reshape(1,1))

In [42]:
make_name = 'nissan'
model_name = 'pathfinder'
year = 2017
url = urllib.request.urlopen("https://api.edmunds.com/api/vehicle/v2/"+make_name+"/"+model_name+"/"+str(year)+"/styles?fmt=json&api_key=zmhe9zsjfph8rmpepqs97ay3&view=full")
spec_test = json.loads(url.read())
spec_table = pd.DataFrame(spec_test['styles'])
spec_table['model_name'] = model_name
spec_table['make_name'] = make_name

spec_table_2 = spec_table[['model_name','make_name','trim','price','engine','MPG', 'transmission','drivenWheels','numOfDoors','options','year']]
spec_table_2.index = [model_name +'_'+ str(x) for x in range(len(spec_table_2))]
prcie_test = pd.read_json(spec_table_2['price'].to_json(), orient='index')
engine_test = pd.read_json(spec_table_2['engine'].to_json(), orient='index')
engine_test2 = pd.read_json(engine_test['valve'].to_json(), orient='index')
engine_test4 = pd.read_json(engine_test['rpm'].to_json(), orient='index')
engine_test4.columns = ['rpm_hp','rpm_tq']
engine_test3 = pd.concat([engine_test, engine_test2, engine_test4], axis=1)
MPG_test = pd.read_json(spec_table_2['MPG'].to_json(), orient='index')
transmission_test = pd.read_json(spec_table_2['transmission'].to_json(), orient='index')
year_test = pd.read_json(spec_table_2['year'].to_json(), orient='index')
year_test.columns = ['id','model_year']

final_spec_test = pd.concat([spec_table_2, prcie_test, engine_test3, MPG_test, transmission_test, year_test], axis=1)

final_spec_test = final_spec_test[['model_name','make_name','trim','city','highway','model_year',
                                   'baseInvoice', 'baseMSRP', 'deliveryCharges','usedPrivateParty', 'usedTmvRetail', 'usedTradeIn',
                                   'code', 'compressionRatio','horsepower','torque','type','rpm_hp','rpm_tq',
                                   'transmissionType', 'numberOfSpeeds',
                                   'drivenWheels','numOfDoors','options']]

final_spec_test.columns = ['model_name','make_name','trim','city','highway','model_year',
                                   'baseInvoice', 'baseMSRP', 'deliveryCharges','usedPrivateParty', 'usedTmvRetail', 'usedTradeIn',
                                   'Engine_code', 'compressionRatio','horsepower','torque','gas_type','rpm_hp','rpm_tq',
                                   'transmissionType', 'numberOfSpeeds',
                                   'drivenWheels','numOfDoors','options']

final_spec_test = final_spec_test.sort_values('baseMSRP')

spec_table = pd.concat([final_spec_test,pre_dummy])
pre_dummy = spec_table

In [43]:
spec_table['model_name'].unique()

array(['pathfinder', 'highlander', 'pilot', 'santa-fe', nan], dtype=object)

In [46]:
spec_table = spec_table.drop(0)
spec_table = spec_table.drop([0],axis=1)

In [48]:
spec_table.to_csv('my_suv_spec.csv')
spec_table.to_json('my_suv_spec.json')

In [22]:
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\spec\\픽업트럭')